In [1]:
## Script to split the data into a federated Dataset
## Code from the same partitiondata.py file that was submitted in Part 1
## This will just create the dataset directory in the structure that we want and load the images

import numpy as np
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import os



def split_data(dataset, num_clients, deviation_factor=0.1):
    """
    Splits data across clients with slight deviation
    """
    client_indices = {i: [] for i in range(num_clients)}

    # Split indices based on classes
    for class_idx in range(10):
        class_indices = np.where(np.array(dataset.targets) == class_idx)[0]
        np.random.shuffle(class_indices)

        # Calculate the number of samples
        total_samples = len(class_indices)
        avg_samples_per_client = total_samples // num_clients


        splits = []
        remaining_samples = total_samples
        for client in range(num_clients):
            # Apply deviation to the average samples
            deviation = np.random.randint(-int(avg_samples_per_client * deviation_factor),
                                          int(avg_samples_per_client * deviation_factor) + 1)
            client_sample_count = avg_samples_per_client + deviation

            if client == num_clients - 1:
                client_sample_count = remaining_samples
            else:
                client_sample_count = min(client_sample_count, remaining_samples)

            remaining_samples -= client_sample_count
            splits.append(client_sample_count)

        split_class_indices = np.split(class_indices, np.cumsum(splits[:-1]))

        for client, indices in enumerate(split_class_indices):
            client_indices[client].extend(indices)

    return client_indices



# Function to save client data as jpg
def save_client_data_as_jpg(dataset, indices, client_dir, client_id):
    client_path = os.path.join(client_dir, f'client_{client_id}')
    os.makedirs(client_path, exist_ok=True)


    for idx in indices:
        img, label = dataset[idx]
        label_folder = os.path.join(client_path, f'label_{label}')
        os.makedirs(label_folder, exist_ok=True)

        # Save the image as a .jpg
        img_path = os.path.join(label_folder, f'{idx}.jpg')

        img_pil = transforms.ToPILImage()(img)
        img_pil.save(img_path)





def main():
    transform = transforms.Compose([
                                transforms.RandomHorizontalFlip(p=0.3),
                                transforms.RandomApply([transforms.RandomRotation(10)], p=0.3),
                                transforms.RandomResizedCrop(32, scale=(0.8, 1.0)),
                                # transforms.RandomApply([transforms.GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 2.0))], p=0.2),
                                transforms.ToTensor(),
                                # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                ])

    train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

    num_clients = 10
    client_dir = './federated_clients_data/'

    # Spliting data for training and testing
    train_indices = split_data(train_dataset, num_clients, deviation_factor=0.2)
    test_indices = split_data(test_dataset, num_clients, deviation_factor=0.2)

    # Saving the  data for each client
    for client_id in range(num_clients):
        save_client_data_as_jpg(train_dataset, train_indices[client_id], client_dir, client_id)
        save_client_data_as_jpg(test_dataset, test_indices[client_id], client_dir, client_id)

    print(f"Data has been split and saved into '{client_dir}' for {num_clients} clients.")

if __name__ == '__main__':
    main()

100%|██████████| 170498071/170498071 [00:15<00:00, 11174824.06it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Data has been split and saved into './federated_clients_data/' for 10 clients.


In [2]:
### Script to poison one of the clients
import os
import random
from PIL import Image

NUM_CLASSES = 10

def flip_label_per_image(original_label):
    available_labels = list(range(NUM_CLASSES))
    available_labels.remove(original_label)
    new_label = random.choice(available_labels)  # Randomly select a new label from remaining labels
    return new_label

def poison_client_data_per_image(original_dir, poisoned_dir, num_clients_to_poison):

    os.makedirs(poisoned_dir, exist_ok=True)


    client_dirs = [d for d in os.listdir(original_dir) if d.startswith('client_')]

    # Selecting client to poison
    random.shuffle(client_dirs)
    clients_to_poison = client_dirs[:num_clients_to_poison]

    print(f"Poisoning the following clients: {clients_to_poison}")

    for client in clients_to_poison:

        client_path = os.path.join(original_dir, client)


        poisoned_client_path = os.path.join(poisoned_dir, f'poisoned_{client}')
        os.makedirs(poisoned_client_path, exist_ok=True)

        for label_folder in os.listdir(client_path):
            original_label_path = os.path.join(client_path, label_folder)

            if os.path.isdir(original_label_path):
                original_label = int(label_folder.split("_")[1])

                for img_file in os.listdir(original_label_path):
                    img_path = os.path.join(original_label_path, img_file)

                    # Determine the new flipped label for this image
                    new_label = flip_label_per_image(original_label)

                    poisoned_label_path = os.path.join(poisoned_client_path, f'label_{new_label}')
                    os.makedirs(poisoned_label_path, exist_ok=True)

                    img = Image.open(img_path)
                    img.save(os.path.join(poisoned_label_path, img_file))

    print(f"Poisoning complete. Poisoned data saved in: {poisoned_dir}")


original_data_dir = './federated_clients_data'
poisoned_data_dir = './poisoned_clients_data'   # new directory to save poisoned data
num_clients_to_poison = 1  # No of cleints to poison


poison_client_data_per_image(original_data_dir, poisoned_data_dir, num_clients_to_poison)

Poisoning the following clients: ['client_8']
Poisoning complete. Poisoned data saved in: ./poisoned_clients_data


In [ ]:
# These commands will move the poisoned client data to the original directory so that we have one poisoned client
!cd federated_clients_data/
!rm -r client_2
!mv poisoned_client_2 client_2
# Move the directory to the ./federated_clients_data

rm: cannot remove 'client_2': No such file or directory


In [3]:
import torch
import numpy as np
import matplotlib.pyplot as plt

In [4]:
print(torch.__version__)

2.4.1+cu121


In [5]:
!pip install -q flwr[simulation] flwr-datasets[vision]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 478.6/478.6 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not curren

In [6]:
from collections import OrderedDict
from typing import List, Tuple

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from datasets.utils.logging import disable_progress_bar
from torch.utils.data import DataLoader

import flwr
from flwr.client import Client, ClientApp, NumPyClient
from flwr.common import Metrics, Context
from flwr.server import ServerApp, ServerConfig, ServerAppComponents
from flwr.server.strategy import FedAvg
from flwr.simulation import run_simulation
from flwr_datasets import FederatedDataset

DEVICE = torch.device("cpu")  # Try "cuda" to train on GPU
print(f"Training on {DEVICE}")
print(f"Flower {flwr.__version__} / PyTorch {torch.__version__}")
disable_progress_bar()

Training on cpu
Flower 1.12.0 / PyTorch 2.4.1+cu121


In [7]:
NUM_CLIENTS = 10
BATCH_SIZE = 32

In [8]:
import os
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image

In [9]:
## Custom Dataset class that reads the data for the client from the appropriate directory based on the client ID provided
class ClientDataset(Dataset):
  def __init__(self,root_dir,client_id,transform=None):
    self.client_dir = os.path.join(root_dir,client_id)
    self.transform = transform
    self.images = []
    self.labels = []
    self._load_data()
  def _load_data(self):
    for label in os.listdir(self.client_dir):
            label_dir = os.path.join(self.client_dir, label)
            if os.path.isdir(label_dir):
                for img_file in os.listdir(label_dir):
                    img_path = os.path.join(label_dir, img_file)
                    self.images.append(img_path)
                    self.labels.append(int(label[-1]))

  def __len__(self):
    return len(self.images)

  def __getitem__(self, idx):
    img_path = self.images[idx]
    image = Image.open(img_path)
    label = self.labels[idx]
    if self.transform:
        image = self.transform(image)
    return image, label


In [10]:
## Function that is called by a Flower Framework Client to load the dataset
## Uses the Client Dataset class created above to using the specified client id derived from the partition id
def load_datasets(partition_id:int):
    root_dir = "/content/federated_clients_data"
    client_id = "client_" + str(partition_id)
    transform = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize to match CIFAR-10 dimensions
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    client_dataset = ClientDataset(root_dir=root_dir, client_id=client_id, transform=transform)

    train_size = int(0.8 * len(client_dataset))
    test_size = len(client_dataset) - train_size

    train_dataset, test_dataset = random_split(client_dataset, [train_size, test_size])

    train_size = int(0.8 * len(train_dataset))
    val_size = len(train_dataset) - train_size
    train_dataset, val_dataset = random_split(train_dataset,[train_size,val_size])
    trainloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    valloader = DataLoader(val_dataset,batch_size=BATCH_SIZE,shuffle=False)
    testloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    return trainloader,valloader,testloader




In [11]:
## Defines the neural network architecture that would be used by all the clients
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # convolutional layer (sees 32x32x3 image tensor)
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        # convolutional layer (sees 16x16x16 tensor)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        # convolutional layer (sees 8x8x32 tensor)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        # max pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        # linear layer (64 * 4 * 4 -> 500)
        self.fc1 = nn.Linear(64 * 4 * 4, 500)
        # linear layer (500 -> 10)
        self.fc2 = nn.Linear(500, 10)
        # dropout layer (p=0.25)
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        # add sequence of convolutional and max pooling layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        # flatten image input
        x = x.view(-1, 64 * 4 * 4)
        # add dropout layer
        x = self.dropout(x)
        # add 1st hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        # add dropout layer
        x = self.dropout(x)
        # add 2nd hidden layer, with relu activation function
        x = self.fc2(x)
        return x

In [12]:
import torch.optim as optim

In [13]:
# Train and Test functions that are used by clients fit and evaluate methods
def train(net, trainloader, epochs: int, verbose=False):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.01)
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for batch in trainloader:
            images, labels = batch
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss.item()*labels.size(0)
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        if verbose:
            print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")


def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for batch in testloader:
            images, labels = batch
            outputs = net(images)
            loss += criterion(outputs, labels).item()*labels.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total

    return loss, accuracy

In [14]:
# Functions to get and set parameters for the model on clients
def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]

In [15]:
# Global dictionaries to track the metrics across rounds for each client
accuracy_per_client_per_round = {}
loss_per_client_per_round = {}

In [16]:
# Defines a flower client class
class FlowerClient(NumPyClient):
    def __init__(self, net, trainloader, valloader,partitionId):
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader
        self.id = partitionId
        self.metrics_list = []

    def get_parameters(self, config):
        return get_parameters(self.net)

    def fit(self, parameters, config):
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        self.metrics_list.append((loss,accuracy))
        return float(loss), len(self.valloader), {"accuracy": float(accuracy),"loss": float(loss),"partition":str(self.id)}

In [17]:
## A global cache to store the clients
clients_cache = {}

In [18]:
# Function to create clients that will passed to the simulation so that the server can create clients as required
def client_fn(context: Context) -> Client:
    """Create a Flower client representing a single organization."""

    # Load model
    net = Net().to(DEVICE)

    # Load data (CIFAR-10)
    # Note: each client gets a different trainloader/valloader, so each client
    # will train and evaluate on their own unique data partition
    # Read the node_config to fetch data partition associated to this node
    partition_id = context.node_config["partition-id"]
    if partition_id not in clients_cache:
      trainloader, valloader, _ = load_datasets(partition_id=partition_id)
      clients_cache[partition_id] = FlowerClient(net, trainloader, valloader,partition_id).to_client()

    # Create a single Flower client representing a single organization
    # FlowerClient is a subclass of NumPyClient, so we need to call .to_client()
    # to convert it to a subclass of `flwr.client.Client`
    return clients_cache[partition_id]


# Create the ClientApp
client = ClientApp(client_fn=client_fn)

In [19]:
# Custom Strategy Class to store metrics for each round into the global dictonaries
from flwr.server.strategy import FedAvg

class CustomStrategy(FedAvg):
  def aggregate_evaluate(self,rnd,results,failure):
    if failure:
      print(f"Round {rnd} had {len(failure)} failures")
    accuracy_list = [r.metrics["accuracy"] for _, r in results]

    for _,r in results:
      if r.metrics["partition"] not in accuracy_per_client_per_round:
        accuracy_per_client_per_round[r.metrics["partition"]] = []
      accuracy_per_client_per_round[r.metrics["partition"]].append((rnd,r.metrics["accuracy"]))
      if r.metrics["partition"] not in loss_per_client_per_round:
        loss_per_client_per_round[r.metrics["partition"]] =[]
      loss_per_client_per_round[r.metrics["partition"]].append((rnd,r.metrics["loss"]))

    average_accuracy = sum(accuracy_list) / len(accuracy_list)
    print(f"Round {rnd} accuracy: {average_accuracy}")
    print(f"Accuracy List for Round {rnd} : {accuracy_list}")


    return super().aggregate_evaluate(rnd, results, failure)

In [20]:
# Specify the resources each of your clients need
# By default, each client will be allocated 1x CPU and 0x GPUs
backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 0.0}}

if DEVICE.type == "cuda":
    backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 1.0}}

In [21]:
# Global list for storing aggreagated metric
aggregate_metric_list = []

In [22]:
# Function that calculates the average loss and accuracy
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    losses = [num_examples * m["loss"] for num_examples, m in metrics]  # For weighted loss
    examples = [num_examples for num_examples, _ in metrics]
    accuracy = sum(accuracies) / sum(examples)
    loss = sum(losses) / sum(examples)
    aggregate_metric_list.append({"accuracy": accuracy, "loss": loss})
    # Aggregate and return custom metric (weighted average)
    return {"accuracy": accuracy, "loss": loss}

In [23]:
# Function that creates the Server object contains the Custom Stategy mentioned above
def server_fn(context: Context) -> ServerAppComponents:
    """Construct components that set the ServerApp behaviour.
    """

    strategy = CustomStrategy(
        fraction_fit=1.0,
        fraction_evaluate=1.0,
        min_fit_clients=10,
        min_evaluate_clients=10,
        min_available_clients=10,
        evaluate_metrics_aggregation_fn=weighted_average,  # <-- pass the metric aggregation function
    )

    # Configure the server for 100 rounds of training
    config = ServerConfig(num_rounds=100)

    return ServerAppComponents(strategy=strategy, config=config)

In [24]:
# Running the simulation
import os
os.environ["RAY_DEDUP_LOGS"] = "0"

# Create a new server instance with the updated FedAvg strategy
server = ServerApp(server_fn=server_fn)

# Run simulation
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_CLIENTS,
    backend_config=backend_config,
)

print(accuracy_per_client_per_round)
print(loss_per_client_per_round)
print(aggregate_metric_list)

DEBUG:flwr:Asyncio event loop already running.
INFO :      Starting Flower ServerApp, config: num_rounds=100, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=2064) 2024-10-15 23:47:22.130876: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=2064) 2024-10-15 23:47:22.214978: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=2064) 2024-10-15 23:47:22.248748: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(pid=2064) 2024-10-15 23:47:24.499198: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warnin

Round 1 accuracy: 0.10740151576905403
Accuracy List for Round 1 : [0.12021857923497267, 0.12044534412955465, 0.10899470899470899, 0.12796697626418987, 0.10961737331954498, 0.0950920245398773, 0.08981380065717415, 0.09478168264110756, 0.10618556701030928, 0.1008991008991009]


INFO :      aggregate_fit: received 10 results and 0 failures
INFO :      configure_evaluate: strategy sampled 10 clients (out of 10)
INFO :      aggregate_evaluate: received 10 results and 0 failures
INFO :      
INFO :      [ROUND 3]
INFO :      configure_fit: strategy sampled 10 clients (out of 10)


Round 2 accuracy: 0.11523267787460453
Accuracy List for Round 2 : [0.091005291005291, 0.10237849017580145, 0.10117145899893504, 0.1134020618556701, 0.14098360655737704, 0.12280701754385964, 0.1114519427402863, 0.08652792990142388, 0.14473684210526316, 0.13786213786213786]


INFO :      aggregate_fit: received 10 results and 0 failures
INFO :      configure_evaluate: strategy sampled 10 clients (out of 10)
INFO :      aggregate_evaluate: received 10 results and 0 failures
INFO :      
INFO :      [ROUND 4]
INFO :      configure_fit: strategy sampled 10 clients (out of 10)


...

Round 99 accuracy: 0.5269743039660977
Accuracy List for Round 99 : [0.5698035160289555, 0.5964912280701754, 0.5846994535519126, 0.5748987854251012, 0.5724275724275725, 0.5777777777777777, 0.5876288659793815, 0.05147864184008762, 0.5869120654396728, 0.5676251331203408]


INFO :      aggregate_fit: received 10 results and 0 failures
INFO :      configure_evaluate: strategy sampled 10 clients (out of 10)
INFO :      aggregate_evaluate: received 10 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 100 round(s) in 5923.35s
INFO :      	History (loss, distributed):
INFO :      		round 1: 2.3016450618755444
INFO :      		round 2: 2.300528020362186
INFO :      		round 3: 2.2986938815798035
INFO :      		round 4: 2.2967706643252748
INFO :      		round 5: 2.2936485564378146
INFO :      		round 6: 2.287000250007291
INFO :      		round 7: 2.273859418352853
INFO :      		round 8: 2.251391642802008
INFO :      		round 9: 2.2174513182493074
INFO :      		round 10: 2.1790716063501963
INFO :      		round 11: 2.1428780621056203
INFO :      		round 12: 2.1135841257290613
INFO :      		round 13: 2.0825779749854614
INFO :      		round 14: 2.0656052036590618
INFO :      		round 15: 2.070693398674157
INFO :      		round 16: 2.032491386898954

Round 100 accuracy: 0.5352368862986969
Accuracy List for Round 100 : [0.5793814432989691, 0.5931174089068826, 0.5967213114754099, 0.5894105894105894, 0.5687693898655636, 0.04928806133625411, 0.6042944785276073, 0.617678381256656, 0.5727554179566563, 0.580952380952381]


INFO :      		round 94: 1.4993822089812032
INFO :      		round 95: 1.4988742280326994
INFO :      		round 96: 1.4574257682796006
INFO :      		round 97: 1.4803713150636377
INFO :      		round 98: 1.4806113527432272
INFO :      		round 99: 1.4584128635209461
INFO :      		round 100: 1.4520338120001668
INFO :      	History (metrics, distributed, evaluate):
INFO :      	{'accuracy': [(1, 0.10744763225536309),
INFO :      	              (2, 0.11545177854448432),
INFO :      	              (3, 0.11063268182984866),
INFO :      	              (4, 0.13205497459530796),
INFO :      	              (5, 0.16426603985780908),
INFO :      	              (6, 0.1843360012195907),
INFO :      	              (7, 0.19491544432613434),
INFO :      	              (8, 0.18593524757275345),
INFO :      	              (9, 0.20930678648155973),
INFO :      	              (10, 0.22059119324693585),
INFO :      	              (11, 0.22186926719128974),
INFO :      	              (12, 0.2357267388786322),
INFO :

{'9': [(1, 0.12021857923497267), (2, 0.14098360655737704), (3, 0.14754098360655737), (4, 0.18688524590163935), (5, 0.21092896174863388), (6, 0.24153005464480876), (7, 0.2448087431693989), (8, 0.23825136612021858), (9, 0.24918032786885247), (10, 0.24262295081967214), (11, 0.26666666666666666), (12, 0.27759562841530055), (13, 0.2939890710382514), (14, 0.2754098360655738), (15, 0.3005464480874317), (16, 0.3224043715846995), (17, 0.32349726775956283), (18, 0.3366120218579235), (19, 0.34863387978142074), (20, 0.3836065573770492), (21, 0.3442622950819672), (22, 0.37486338797814206), (23, 0.3453551912568306), (24, 0.40327868852459015), (25, 0.40327868852459015), (26, 0.3628415300546448), (27, 0.4273224043715847), (28, 0.45027322404371584), (29, 0.45027322404371584), (30, 0.4371584699453552), (31, 0.4546448087431694), (32, 0.45792349726775955), (33, 0.45136612021857925), (34, 0.46120218579234973), (35, 0.4644808743169399), (36, 0.4448087431693989), (37, 0.46994535519125685), (38, 0.45901639344

In [25]:
import pickle

In [26]:
# Saving the metric files to disk using pickle
with open('poisoned_accuracy_per_client_per_round.pkl', 'wb') as f:
    pickle.dump(accuracy_per_client_per_round, f)
with open('poisoned_loss_per_client_per_round.pkl','wb') as f:
    pickle.dump(loss_per_client_per_round,f)
with open('poisoned_aggregate_metric_list.pkl','wb') as f:
    pickle.dump(aggregate_metric_list,f)

In [ ]:
# Client 8 was poisoned in this case